<a href="https://colab.research.google.com/github/amyhu0114/CS315-project2-group2/blob/main/Filtered_posts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import os, types
import re
import csv
from google.colab import files
from pandas import Series, DataFrame
import pandas as pd
import dateutil.parser as dparser
from random import randint

In [53]:
#gets the csv file of news accounts we wrote
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
hashtagsList = "/content/drive/MyDrive/hashtags.csv"

In [55]:
#saves the hashtags into a list
newsHashtags = []
with open(hashtagsList, 'r') as file:
  csv_reader = csv.reader(file)
  for row in csv_reader:
    if row:  # Check if row is not empty
      newsHashtags.append(row[0])

newsHashtags

['Hashtag name',
 'uspolitics',
 'republicans',
 'republican',
 'democrat',
 'democrats',
 'liberal',
 'politics',
 'politicaltiktok',
 '2024election',
 '2024president',
 'news',
 'breakingnews',
 'conservative',
 'immigration',
 'bordercrisis',
 'ukraine',
 'palestine',
 'gaza',
 'election',
 'trump',
 'biden',
 'covid',
 'economy',
 'breakingnews',
 'journalism',
 'worldnews',
 'economics',
 'FederalReserve',
 'leftist',
 'liberalism',
 'trump2024',
 'newstiktok',
 '2024election',
 'newsattiktok',
 'noticiastiktok',
 'noticiasen1minuto',
 'noticias',
 'newsupdate',
 'inflation',
 'chinesepolitics',
 'climatechange',
 'climategoals',
 'currentnews',
 'governmentshutdown',
 'learnontiktok',
 'trumptrial',
 'historytok',
 'trumprally',
 'nato',
 'trump',
 'crime',
 'michigan',
 'conservative',
 'rightwing',
 'vote',
 'fake news',
 'ballot',
 'israel',
 'usapolitics',
 'usaelections',
 'uselections',
 'joebiden',
 '2024debate',
 'hamas',
 'crimetok',
 'breaking',
 'climate',
 'supremecou

In [56]:
#uploads the csv files of pyktok results
files.upload()

Saving results_33534.csv to results_33534 (1).csv


{'results_33534 (1).csv': b'video_id,video_timestamp,video_duration,video_locationcreated,suggested_words,video_diggcount,video_sharecount,video_commentcount,video_playcount,video_description,video_is_ad,video_stickers,author_username,author_name,author_followercount,author_followingcount,author_heartcount,author_videocount,author_diggcount,author_verified\n7315561816673750318,2023-12-22T18:16:20,18.0,US,"flipnote, flipnote animation, asdf movie, flipnote animation horror, flipnotes nintendo dsi, flipnote in 2023, flip note animation tutorial, animation, flipnote animation old, ds flip notes",3600000.0,40100.0,11000.0,19000000.0,Bonus flipnote for you,False,,zanelittlemusic,Zane Little,,,,,,False\n7331462048699649323,2024-02-03T14:37:35,8.0,US,,1111.0,1.0,2.0,6589.0,\xe2\x98\x81\xef\xb8\x8f\xf0\x9f\x93\x99,False,,createbyua,ua,,,,,,False\n7318253025649315104,2023-12-30T00:21:20,6.0,TH,,2208.0,3.0,8.0,42400.0,at 30 & 25 years of age \xf0\x9f\x98\x82\xf0\x9f\x98\x82,False,,confusedindian

In [57]:
data = pd.read_csv("results_33534.csv")
posts = pd.DataFrame(data)
#removing nan values
posts = posts[posts['video_description'].notna()]
#filtering to videos in the US
posts = posts[posts['video_locationcreated'] == 'US']
posts.head()

,video_id,video_timestamp,video_duration,video_locationcreated,suggested_words,video_diggcount,video_sharecount,video_commentcount,video_playcount,video_description,video_is_ad,video_stickers,author_username,author_name,author_followercount,author_followingcount,author_heartcount,author_videocount,author_diggcount,author_verified
0,7315561816673750318,2023-12-22T18:16:20,18.0,US,"flipnote, flipnote animation, asdf movie, flip...",3600000.0,40100.0,11000.0,19000000.0,Bonus flipnote for you,False,NaN,zanelittlemusic,Zane Little,NaN,NaN,NaN,NaN,NaN,False
1,7331462048699649323,2024-02-03T14:37:35,8.0,US,NaN,1111.0,1.0,2.0,6589.0,☁️📙,False,NaN,createbyua,ua,NaN,NaN,NaN,NaN,NaN,False
3,7331462048699649323,2024-02-03T14:37:35,8.0,US,NaN,1111.0,1.0,2.0,6589.0,☁️📙,False,NaN,createbyua,ua,NaN,NaN,NaN,NaN,NaN,False
5,7331462048699649323,2024-02-03T14:37:35,8.0,US,NaN,1111.0,1.0,2.0,6589.0,☁️📙,False,NaN,createbyua,ua,NaN,NaN,NaN,NaN,NaN,False
7,7322969142024342826,2024-01-11T17:20:34,5.0,US,NaN,196.0,0.0,7.0,5198.0,🥴😅👍🏾 #vegan #chipotle #veggie #food,False,NaN,darlene_octavia,darleneoctavia,NaN,NaN,NaN,NaN,NaN,False


In [58]:
def getHashtags(text):
  """Takes a string phrase and returns hashtags if present"""
  if isinstance(text, str):
    return re.findall(r"#(\S+)", text.lower())
  return []

In [59]:
#extracts the hashtags for each post
posts['hashtags'] = posts['video_description'].apply(getHashtags)
#remove posts with no hashtags
posts = posts[posts['hashtags'].map(lambda x: len(x)) > 0]
posts.head()

,video_id,video_timestamp,video_duration,video_locationcreated,suggested_words,video_diggcount,video_sharecount,video_commentcount,video_playcount,video_description,...,video_stickers,author_username,author_name,author_followercount,author_followingcount,author_heartcount,author_videocount,author_diggcount,author_verified,hashtags
7,7322969142024342826,2024-01-11T17:20:34,5.0,US,NaN,196.0,0.0,7.0,5198.0,🥴😅👍🏾 #vegan #chipotle #veggie #food,...,NaN,darlene_octavia,darleneoctavia,NaN,NaN,NaN,NaN,NaN,False,"[vegan, chipotle, veggie, food]"
8,7300994079754636587,2023-11-13T12:06:02,60.0,US,NaN,20700.0,16.0,33.0,139300.0,Just wait until you see his trick #interview #...,...,NaN,charxstyle,char,NaN,NaN,NaN,NaN,NaN,False,"[interview, food, bestfriend, charxstyle]"
9,7322969142024342826,2024-01-11T17:20:34,5.0,US,NaN,196.0,0.0,7.0,5198.0,🥴😅👍🏾 #vegan #chipotle #veggie #food,...,NaN,darlene_octavia,darleneoctavia,NaN,NaN,NaN,NaN,NaN,False,"[vegan, chipotle, veggie, food]"
13,7309956665221205291,2023-12-07T15:45:27,9.0,US,NaN,10300.0,59.0,91.0,413500.0,Understanding your seasonal color palette is k...,...,NaN,jacquie.lm,Jacquie,NaN,NaN,NaN,NaN,NaN,False,"[seasonalcoloranalysis, coloranalysis, styling..."
17,7326989936077458731,2024-01-22T13:23:51,5.0,US,NaN,10600.0,437.0,128.0,73100.0,"the way my entire class screamed, live laugh l...",...,NaN,happylilflower,happylilflower,NaN,NaN,NaN,NaN,NaN,False,"[badbunny, nadiesabeloquevaapasarmañana, regga..."


In [60]:
#a smaller dataframe with just the needed functions
postsSmall = posts[["video_id", "author_username", "video_description",
                "hashtags", "suggested_words"]].copy()
postsSmall.head()

,video_id,author_username,video_description,hashtags,suggested_words
7,7322969142024342826,darlene_octavia,🥴😅👍🏾 #vegan #chipotle #veggie #food,"[vegan, chipotle, veggie, food]",NaN
8,7300994079754636587,charxstyle,Just wait until you see his trick #interview #...,"[interview, food, bestfriend, charxstyle]",NaN
9,7322969142024342826,darlene_octavia,🥴😅👍🏾 #vegan #chipotle #veggie #food,"[vegan, chipotle, veggie, food]",NaN
13,7309956665221205291,jacquie.lm,Understanding your seasonal color palette is k...,"[seasonalcoloranalysis, coloranalysis, styling...",NaN
17,7326989936077458731,happylilflower,"the way my entire class screamed, live laugh l...","[badbunny, nadiesabeloquevaapasarmañana, regga...",NaN


In [61]:
#check if any posts have our hashtags
newsDF = postsSmall[postsSmall['hashtags'].apply(lambda htlst: any(hashtag in htlst for hashtag in newsHashtags)
                              if isinstance(htlst, list) else False)]
newsDF.head()

,video_id,author_username,video_description,hashtags,suggested_words
162,7338698880831442222,911access,"Watch Gabriella Wong, founder of accesSOS, pre...","[unitednations, techforgood, zerocon24, access...","access sos, gabrielle wong, access sos app, pu..."
312,7335086838119501086,aljazeeraenglish,'Our military operation in Rafah...' #US Presi...,"[us, joebiden, kingabdullah, news, biden]","our military operation in rafah, biden rafah, ..."
345,7325491869574532395,ssghowardmi,Bucko that lil $4500 a year is NOT going to pa...,"[army, education, miltok, michigan, nationalgu...","army national guard, sat score, sat, College S..."
901,7333685047167438122,multicinema,bellerín dont get me for this one 👀… this vers...,"[hectorbellerin, héctorbellerín, hectorbelleri...","soaked v2, soaked shy smith, soaked edit audio..."
1063,7309179331329183022,newsweek,Replying to @Newsweek #news #newsweek #politic...,"[news, newsweek, politics, congress, higheredu...",NaN
